In [97]:
import os
import pandas as pd
from extraction import FemTransform, FemExtract
import settings
import numpy as np
import func

In [113]:
sip_folder_path = '/Users/ivanov.ev/Work/Dashboard/sip'
sip_data = 'Свод_СИП_2023-2025_v20.xlsx'
sip_meta = 'meta.xlsx'
sip_data_sheet_name = 'Свод'
sip_meta_sheet_name = 'meta'
sip_data_path = os.path.join(sip_folder_path, sip_data)
sip_meta_path = os.path.join(sip_folder_path, sip_meta)

In [144]:
sip_data_file = pd.read_excel(sip_data_path, sheet_name=sip_data_sheet_name)
sip_meta_file = pd.read_excel(sip_meta_path, sheet_name=sip_meta_sheet_name)
df = sip_data_file
columns = sip_meta_file

df.columns = list(df.loc[0,])
df = df.loc[1:,]
columns = columns.dropna()
rename = columns.set_index([columns.columns.to_list()[0]]).to_dict()[columns.columns.to_list()[1]]
df = df.rename(columns=rename)[list(rename.values())]

categorical_data = ['programme', 'code', 'project', 'investment object', 'allocation', 'criticality', 'sanctions', 'expert_opinion', 'effect_type', 'investment_category', 'investment_goal', 'scenario', 'stage', 'bucket', 'platform', 'architecture_appraisal_platform', 'dbms', 'architecture_appraisal_dbms', 'operational_system', 'architecture_appraisal_os', 'infrastructure', 'architecture_appraisal_inf', 'in_prod', 'portfel', 'circulation', 'challenge_2022']
metric_data = ['factcapexopex_2022', 'capex_2022', 'capex_2022/1', 'capex_2023/2','capex_2024', 'capex_2025', 'capex_2026', 'capex_2027', 'opex_2022', 'opex_2023/1', 'opex_2023/2', 'opex_2024', 'opex_2025', 'opex_2026','opex_2027', 'npv_2022', 'npv_2023', 'npv_2024', 'npv_2025', 'npv_2026','npv_2027', 'npv_2028', 'npv_2029', 'npv_2030', 'umv_2022', 'umv_2023','umv_2024', 'umv_2025', 'umv_2026', 'umv_2027', 'umv_2028', 'umv_2029','umv_2030', 'dmv_2022', 'dmv_2023', 'dmv_2024', 'dmv_2025', 'dmv_2026', 'dmv_2027', 'dmv_2028', 'dmv_2029', 'dmv_2030', 'capexinvestmentdecision_2023', 'opexinvestmentdecision_2023', 'budgeteconomy_2023']

id_vars = categorical_data
value_vars = list(set(df.columns) - set(categorical_data))
check_list = id_vars + value_vars
print('Without data missing - ', set(df.columns.to_list()) == set(check_list))

# df = pd.melt(df, id_vars=id_vars, var_name='metrics', value_name='value', )
# df['metrics'] = df['metrics'].astype('int')
df = df.melt(id_vars=categorical_data,
             value_vars=value_vars,
             var_name='metrics',
             value_name='value',
             ignore_index=False)
df[['metric', 'year']] = df['metrics'].str.split('_', expand=True)
df = df.drop(columns=['metrics'])
df

Without data missing -  True


,programme,code,project,investment object,allocation,criticality,sanctions,expert_opinion,effect_type,investment_category,...,architecture_appraisal_os,infrastructure,architecture_appraisal_inf,in_prod,portfel,circulation,challenge_2022,value,metric,year
1,Когнитивная Геология,BI.18.002910_00,Управление реализацией программы и ее компонен...,Проект,КЦ,Управление программой,Управление программой,Управление программой,без эффекта,Уверенные,...,NaN,NaN,NaN,Нет,Да,Нет,Нет,NaN,umv,2029
2,Когнитивная Геология,BI.18.002910_00,Управление реализацией программы и ее компонен...,Проект,КЦ,Управление программой,Управление программой,Управление программой,без эффекта,Уверенные,...,NaN,NaN,NaN,Нет,Да,Нет,Нет,NaN,umv,2029
3,Когнитивная Геология,BI.18.002910_00,Управление реализацией программы и ее компонен...,Проект,КЦ,Управление программой,Управление программой,Управление программой,без эффекта,Приоритетные,...,NaN,NaN,NaN,Нет,Да,Нет,Нет,NaN,umv,2029
4,Когнитивная Геология,U180001199,Виртуальная лаборатория,Проект,КЦ,Важная,Жёлтый,Более 12-ти месяцев,без эффекта,Уверенные,...,Жёлтый,IaaS ГПН,Зелёный,Да,Да,Нет,Нет,NaN,umv,2029
5,Когнитивная Геология,U180001199,Виртуальная лаборатория,Проект,КЦ,Важная,Жёлтый,Более 12-ти месяцев,без эффекта,Приоритетные 1,...,Жёлтый,IaaS ГПН,Зелёный,Да,Да,Нет,Нет,NaN,umv,2029
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,Задание на обрисковку ППОФ (6+6),б/н,Задание на обрисковку ППОФ (6+6),Челлендж,ЭС,NaN,NaN,NaN,без эффекта,Уверенные,...,NaN,NaN,NaN,NaN,Да,Да,Нет,NaN,umv,2027
1269,Задание на обрисковку ППОФ (6+6),б/н,Задание на обрисковку ППОФ (6+6),Челлендж,Ямал,NaN,NaN,NaN,без эффекта,Уверенные,...,NaN,NaN,NaN,NaN,Да,Да,Нет,NaN,umv,2027
1270,Задание на обрисковку ППОФ (6+6),б/н,Задание на обрисковку ППОФ (6+6),Челлендж,Автоматизация,NaN,NaN,NaN,без эффекта,Уверенные,...,NaN,NaN,NaN,NaN,Да,Да,Нет,NaN,umv,2027
1271,Задание на обрисковку проектов HSE,б/н,Задание на обрисковку проектов HSE,Челлендж,КЦ,NaN,NaN,NaN,без эффекта,Уверенные,...,NaN,NaN,NaN,NaN,Да,Нет,Нет,NaN,umv,2027


In [149]:
a = pd.ExcelFile(sip_data_path).parse()

In [151]:
a

,Общая информация,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144,Unnamed: 145
0,Аллокация,Программа,Объект инвестиций:\n- Проект\n- Система Run в ...,Проект,Код ИСУП,Критичность:\n- Критичная - соответствует пред...,Санкционность\n(ИТОГО)\n- Зелёный\n- Жёлтый\n-...,Экспертное мнение по сроку отказа\n- До 3-х ме...,Тип эффекта\n- прямой расчёт (NPV)\n- косвенны...,Категория инвестиций\n- Обязательные\n- Уверен...,...,Сумма 2023,Тираж в ДО?,АБ?,CAPEX 2023,OPEX 2023,J для ДЦТ,Челлендж 2022?,Категория\nпроектов,"Экономия бюджета\n2023 год, тыс. руб",NaN
1,КЦ,Когнитивная Геология,Проект,Управление реализацией программы и ее компонен...,BI.18.002910_00,Управление программой,Управление программой,Управление программой,без эффекта,Уверенные,...,0,Нет,Нет,0,0,0,Нет,NaN,NaN,NaN
2,КЦ,Когнитивная Геология,Проект,Управление реализацией программы и ее компонен...,BI.18.002910_00,Управление программой,Управление программой,Управление программой,без эффекта,Уверенные,...,54097.8,Нет,Нет,0,54097.8,0,Нет,NaN,NaN,NaN
3,КЦ,Когнитивная Геология,Проект,Управление реализацией программы и ее компонен...,BI.18.002910_00,Управление программой,Управление программой,Управление программой,без эффекта,Приоритетные,...,13372.2,Нет,Нет,0,13372.2,0,Нет,NaN,NaN,NaN
4,КЦ,Когнитивная Геология,Проект,Виртуальная лаборатория,U180001199,Важная,Жёлтый,Более 12-ти месяцев,без эффекта,Уверенные,...,0,Нет,Нет,0,0,0,Нет,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1268,ЭС,Задание на обрисковку ППОФ (6+6),Челлендж,Задание на обрисковку ППОФ (6+6),б/н,NaN,NaN,NaN,без эффекта,Уверенные,...,0,Да,Нет,0,0,0,Нет,NaN,NaN,NaN
1269,Ямал,Задание на обрисковку ППОФ (6+6),Челлендж,Задание на обрисковку ППОФ (6+6),б/н,NaN,NaN,NaN,без эффекта,Уверенные,...,-17547.631302,Да,Нет,-17547.631302,0,0,Нет,NaN,NaN,NaN
1270,Автоматизация,Задание на обрисковку ППОФ (6+6),Челлендж,Задание на обрисковку ППОФ (6+6),б/н,NaN,NaN,NaN,без эффекта,Уверенные,...,-90000,Да,Нет,-90000,0,0,Нет,NaN,NaN,NaN
1271,КЦ,Задание на обрисковку проектов HSE,Челлендж,Задание на обрисковку проектов HSE,б/н,NaN,NaN,NaN,без эффекта,Уверенные,...,-57604.52981,Нет,Нет,-40788,-16816.52981,0,Нет,NaN,NaN,NaN
